In [1]:
import numpy as np
import random
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, Model


In [2]:
from google.colab import files
uploaded=files.upload()

Saving blend_data_MOST_UPDATEDD.xlsx to blend_data_MOST_UPDATEDD.xlsx


In [3]:
data=pd.read_excel("blend_data_MOST_UPDATEDD.xlsx")

In [4]:
data.shape

(410, 31)

In [5]:
api_data = {
  'API': [1,2,3,4,5,6,7,8,9,10],
  'API_d10': [3.1, 2.5, 3, 8.01, 10.6, 9.5, 9.1, 2.5, 2.42, 2.5],
  'API_d50': [20, 6.9, 8, 28.5, 35.2, 41, 52.2, 10, 8.4, 10],
  'API_d90': [82, 15, 19.3, 58.7, 65.1, 98, 135, 27.8, 27.6, 27.8],
  'API_d32': [7.3, 4.5, 5.5, 12.1, 14.1, 19.3, 15.5, 5.25, 5.2, 5.25],
  'API_surf_eng': [0.041, 0.039545, 0.04, 0.039, 0.03892, 0.03892, 0.03892, 0.03643, 0.046, 0.03643],
  'API_density': [1290, 1263, 1430, 1120, 1120, 1120, 1120, 1365, 1290, 1365],
  'API AR(-)': [0.55, 0.66, 0.64, 0.6, 0.6, 0.58, 0.59, 0, 0.61, 0],
  'API Sphericity(-)': [0.81, 0.9, 0.84, 0.84, 0.83, 0.82, 0.83, 0, 0.81, 0],
  'API Elongation(-)': [0.45, 0.64, 0.55, 0.49, 0.48, 0.46, 0.47, 0, 0.5, 0]
}
api_reference = pd.DataFrame(api_data)

excipient_data = {
  'excipient': [111,222,333,444,555,666,777,888,999,1111,2222,3333,4444,5555,6666,7777,8888,9999,11111],
  'exc_d10': [0, 21.2, 31.3, 5.6, 53.4, 8, 4.5, 3.65, 48, 1.8, 2.9, 1.9, 25.3, 1.3, 65, 4.5, 3.5, 1.5, 0.88],
  'exc_d50': [0, 64.2, 117, 19.3, 185.8, 14.4, 28.2, 19.2, 115, 8.6, 12.5, 8.5, 93.9, 4.2, 45, 27.9, 21.9, 9.6, 3.2],
  'exc_d90': [0, 143, 165.4, 43.4, 322.3, 22.9, 79.6, 48.6, 202, 20.5, 32.1, 25.5, 163, 22, 103, 88.5, 56.2, 35.4, 7.9],
  'exc_d32': [0, 42.9, 49.1, 10.8, 100.4, 15, 10.1, 6.2, 85.2, 4.29, 6.4, 4.5, 38, 3.23, 11.8, 10.3, 7.3, 4.1, 2.11],
  'exc_surf_eng': [0, 0.03234, 0.056, 0.048, 0.047, 0.03234, 0.042, 0.04469, 0.03948, 0.043, 0.043695, 0.04267, 0.03746, 0.042505, 0.042505, 0.03437, 0.03437, 0.04382, 0.043475],
  'exc_density': [0, 1444, 1563, 1559, 1562, 1444, 1540, 1543, 1543, 1520, 1780, 1710, 1504, 1734, 1734, 1528, 1546, 1642, 1739],
  'exc AR(-)': [0, 0, 0.59, 0.61, 0.73, 0.73, 0.65, 0.65, 0, 0.65, 0, 0.64, 0, 0.62, 0.66, 0.67, 0.67, 0.66, 0.6],
  'exc Sphericity(-)': [0, 0, 0.78, 0.79, 0.86, 0.87, 0.88, 0.83, 0, 0.88, 0, 0.86, 0, 0.85, 0.86, 0.86, 0.88, 0.9, 0.91],
  'exc Elongation(-)': [0, 0, 0.38, 0.45, 0.52, 0.59, 0.6, 0.53, 0, 0.6, 0, 0.58, 0, 0.59, 0.53, 0.55, 0.59, 0.64, 0.71]
}
excipient_reference = pd.DataFrame(excipient_data)

# Specified lists of API and Excipient numbers
api_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
excipient_numbers = [111, 222, 333, 444, 555, 666, 777, 888, 999, 1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 11111]

In [7]:
X = data[['API', 'excipient', 'API_percent', 'API_coat_percent', 'exc_coat_percent', 'API_silica_type', 'exc_silica_type']]
y = data[['FF', 'FF_regime']]


In [8]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming 'X' and 'y' are your features and labels respectively
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Convert 'y' to a numpy array if it's a pandas DataFrame or Series
y = y.to_numpy() if hasattr(y, 'to_numpy') else y

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:

# Convert to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor)).shuffle(buffer_size=1024).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tensor, y_test_tensor)).batch(64)


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras import backend as K

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = K.shape(z_mean)[0]
        dim = K.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

class VAE(Model):
    def __init__(self, input_size, latent_size, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.fc1 = layers.Dense(128, activation='relu')
        self.fc2 = layers.Dense(64, activation='relu')
        self.fc31 = layers.Dense(latent_size)
        self.fc32 = layers.Dense(latent_size)
        self.sampling = Sampling()
        self.fc4 = layers.Dense(64, activation='relu')
        self.fc5 = layers.Dense(128, activation='relu')
        self.fc6 = layers.Dense(input_size + 2)  # Adjusting for FF and FF_regime

    def encode(self, x):
        h1 = self.fc1(x)
        h2 = self.fc2(h1)
        return self.fc31(h2), self.fc32(h2)

    def reparameterize(self, mu, logvar):
        z = self.sampling((mu, logvar))
        return z

    def decode(self, z):
        h3 = self.fc4(z)
        h4 = self.fc5(h3)
        return self.fc6(h4)

    def call(self, inputs):
        mu, logvar = self.encode(inputs)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Example usage
input_size = 7  # Example input size, adjust as necessary
latent_size = 64
vae = VAE(input_size=input_size, latent_size=latent_size)

In [12]:
import tensorflow as tf
from tensorflow.keras import losses

def custom_vae_loss(model, x, reconstructed, additional_outputs, y_true):
    # Reconstruction loss (e.g., MSE for the input data)
    reconstruction_loss = losses.mean_squared_error(x, reconstructed)

    # Additional target prediction loss (e.g., MSE for the additional outputs)
    additional_loss = losses.mean_squared_error(y_true, additional_outputs)

    # KL divergence loss (already added in the model as an added loss)
    kl_loss = sum(model.losses)

    # Total loss
    total_loss = tf.reduce_mean(reconstruction_loss + additional_loss) + kl_loss
    return total_loss


In [55]:
optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(model, x_batch, y_batch, optimizer):
    with tf.GradientTape() as tape:
        x_recon, mu, logvar = model(x_batch, training=True)
        recon_loss = tf.reduce_sum(tf.keras.losses.mse(tf.concat([x_batch, y_batch], axis=1), x_recon))
        kl_div = -0.5 * tf.reduce_sum(1 + logvar - tf.square(mu) - tf.exp(logvar))
        loss = recon_loss + kl_div
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train_vae(model, train_dataset, optimizer, epochs=100):
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataset:
            loss = train_step(model, x_batch, y_batch, optimizer)
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Assuming train_dataset is a tf.data.Dataset object
train_vae(vae, train_dataset, optimizer)


Epoch 0, Loss: 92.0280532836914
Epoch 1, Loss: 39.12620544433594
Epoch 2, Loss: 27.808055877685547
Epoch 3, Loss: 49.96821975708008
Epoch 4, Loss: 20.299957275390625
Epoch 5, Loss: 29.89295196533203
Epoch 6, Loss: 75.5580062866211
Epoch 7, Loss: 45.19984436035156
Epoch 8, Loss: 35.98486328125
Epoch 9, Loss: 31.6015567779541
Epoch 10, Loss: 41.71902847290039
Epoch 11, Loss: 97.74481201171875
Epoch 12, Loss: 42.494075775146484
Epoch 13, Loss: 154.17568969726562
Epoch 14, Loss: 29.130765914916992
Epoch 15, Loss: 48.82012176513672
Epoch 16, Loss: 59.89567184448242
Epoch 17, Loss: 274.880859375
Epoch 18, Loss: 69.36555480957031
Epoch 19, Loss: 94.3967514038086
Epoch 20, Loss: 23.43695831298828
Epoch 21, Loss: 23.786258697509766
Epoch 22, Loss: 39.951019287109375
Epoch 23, Loss: 39.140045166015625
Epoch 24, Loss: 154.12940979003906
Epoch 25, Loss: 28.038164138793945
Epoch 26, Loss: 22.59307861328125
Epoch 27, Loss: 24.7594051361084
Epoch 28, Loss: 28.806978225708008
Epoch 29, Loss: 18.674285

In [15]:
vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  1024      
                                                                 
 dense_1 (Dense)             multiple                  8256      
                                                                 
 dense_2 (Dense)             multiple                  4160      
                                                                 
 dense_3 (Dense)             multiple                  4160      
                                                                 
 sampling (Sampling)         multiple                  0         
                                                                 
 dense_4 (Dense)             multiple                  4160      
                                                                 
 dense_5 (Dense)             multiple                  8320    

In [70]:
predictions = vae.predict(X_test_tensor[:100])
reconstructed_features = predictions[0]

3/3 [==============================] - 0s 7ms/step


In [72]:
original_feature_predictions = reconstructed_features[:, :7]
feature_predictions_denorm = scaler_X.inverse_transform(original_feature_predictions)
ff_predictions = reconstructed_features[:, 7:]

In [73]:

# Reverse the scaling for features
#feature_predictions_denorm = scaler_X.inverse_transform(reconstructed_features)

# Ensure that specific columns are positive
positive_columns_indices = [3, 4, 5, 6]  # Adjust based on your needs
feature_predictions_denorm[:, positive_columns_indices] = np.abs(feature_predictions_denorm[:, positive_columns_indices])

# Proceed with creating the DataFrame and further processing as before
predictions_df = pd.DataFrame(feature_predictions_denorm, columns=['API', 'excipient', 'API_percent', 'API_coat_percent', 'exc_coat_percent', 'API_silica_type', 'exc_silica_type'])
# The rest of your processing...
predictions_df['API'] = [random.choice(api_numbers) for _ in range(len(predictions_df))]
predictions_df['excipient'] = [random.choice(excipient_numbers) for _ in range(len(predictions_df))]

# Assuming api_reference and excipient_reference are DataFrames with reference data
final_result_df = predictions_df.merge(api_reference, on='API').merge(excipient_reference, on='excipient')

# Add FF and FF_regime to the DataFrame
final_result_df['FF'] = ff_predictions[:, 0]
final_result_df['FF_regime'] = ff_predictions[:, 1].round().astype(int)

# Round specified columns to integers
final_result_df[['API_percent', 'API_coat_percent', 'exc_coat_percent', 'API_silica_type', 'exc_silica_type']] = final_result_df[['API_percent', 'API_coat_percent', 'exc_coat_percent', 'API_silica_type', 'exc_silica_type']].round().astype(int)
# Display the first 10 rows of the final DataFrame
final_result_df.head(100)


,API,excipient,API_percent,API_coat_percent,exc_coat_percent,API_silica_type,exc_silica_type,API_d10,API_d50,API_d90,...,exc_d50,exc_d90,exc_d32,exc_surf_eng,exc_density,exc AR(-),exc Sphericity(-),exc Elongation(-),FF,FF_regime
0,3,111,48,38,21,1,0,3.00,8.0,19.3,...,0.0,0.0,0.00,0.000000,0,0.00,0.00,0.00,6.378857,3
1,8,111,53,24,23,1,0,2.50,10.0,27.8,...,0.0,0.0,0.00,0.000000,0,0.00,0.00,0.00,5.950137,3
2,1,111,52,36,13,1,0,3.10,20.0,82.0,...,0.0,0.0,0.00,0.000000,0,0.00,0.00,0.00,27.115372,3
3,1,111,49,37,25,1,0,3.10,20.0,82.0,...,0.0,0.0,0.00,0.000000,0,0.00,0.00,0.00,6.931226,3
4,5,111,56,30,4,1,0,10.60,35.2,65.1,...,0.0,0.0,0.00,0.000000,0,0.00,0.00,0.00,5.599927,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,5,11111,47,40,15,1,0,10.60,35.2,65.1,...,3.2,7.9,2.11,0.043475,1739,0.60,0.91,0.71,12.283597,3
78,8,666,53,35,12,1,0,2.50,10.0,27.8,...,14.4,22.9,15.00,0.032340,1444,0.73,0.87,0.59,15.152763,4
79,4,666,57,40,11,1,0,8.01,28.5,58.7,...,14.4,22.9,15.00,0.032340,1444,0.73,0.87,0.59,5.002381,2
80,8,4444,55,25,9,1,0,2.50,10.0,27.8,...,93.9,163.0,38.00,0.037460,1504,0.00,0.00,0.00,4.475392,2


In [63]:
#predicted_outputs = vae.predict(X_test[:100])

3/3 [==============================] - 0s 3ms/step


In [79]:
# Check if 'y' is a pandas DataFrame
if isinstance(y_test, pd.DataFrame):
    true_column_i = y_test['FF']
    true_column_j = y_test['FF_regime']
elif isinstance(y, np.ndarray):
    true_column_i = y_test[:, 0]
    true_column_j = y_test[:, 1]
else:
    raise TypeError("Unsupported type for 'y'. Expected pandas.DataFrame or numpy.ndarray.")

In [80]:

predicted_column_i = final_result_df['FF']  # Assuming 'predictions' is a DataFrame
predicted_column_j = final_result_df['FF_regime']

In [81]:
from sklearn.metrics import mean_squared_error
# Adjust slicing to ensure the predicted arrays match the size of the true label subsets
predicted_column_i_subset = predicted_column_i[:100]
predicted_column_j_subset = predicted_column_j[:100]
true_column_i_subset = true_column_i[:100]
true_column_j_subset = true_column_j[:100]

# Now, you can calculate MSE with matching lengths
mse_column_i = mean_squared_error(true_column_i_subset, predicted_column_i_subset)
mse_column_j = mean_squared_error(true_column_j_subset, predicted_column_j_subset)

print(f"MSE for Column i: {mse_column_i}")
print(f"MSE for Column j: {mse_column_j}")



MSE for Column i: 172.77876005032587
MSE for Column j: 1.1341463414634145


In [ ]:
final_result_df.to_csv('VAE_model_100_new_columns.csv', index=False)
print("Data saved to 'VAE_model_100_new_columns.csv'")

Data saved to 'VAE_model_100_new_columns.csv'
